In [1]:
import pandas as pd

In [2]:
df = pd.read_excel(r'C:\Users\Mrulay\Desktop\Proposed Solution\Evaluation\Datasets\US2020\US2020.xlsx')

In [3]:
new_header = df.iloc[0] #grab the first row for the header
df = df[1:] #take the data less the header row
df.columns = new_header

In [4]:
import re
from nltk.corpus import stopwords
stop = stopwords.words('english')

def remove_emojis(data):
    emoj = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                      "]+", re.UNICODE)
    return re.sub(emoj, '', data)

def preProcessText(text):
    text = text.lower()
    text = remove_emojis(text)
    text = text.replace(r'[^\w\s]+', '') 
    text = text.replace('@', '')
    text = ' '.join(word.lower() for word in text.split() if word not in stop)
    return text

In [5]:
df.text = df.text.apply(preProcessText)

In [6]:
df

,text,Trump,Biden,West,HOF
1,people follow 18 months ago fired child povert...,Neither,Neutral mentions,Neither,Non-Hateful
2,realdonaldtrump would government stop “preside...,Against,Favor,Neither,Non-Hateful
3,preach brotha! #bidenharris2020 https://t.co/7...,Neither,Favor,Neither,Non-Hateful
4,jrgaillot joebiden class act #gotv #genz #mill...,Neither,Favor,Neither,Non-Hateful
5,"okay white women, 53% made terrible decision 4...",Against,Favor,Neither,Hateful
...,...,...,...,...,...
596,joebiden stay healthy mr vice president! mrs b...,Neither,Favor,Neither,Non-Hateful
597,therock joebiden kamalaharris bye bye rock! it...,Favor,Against,Neither,Non-Hateful
598,countless effortless. [newline][newline]the wa...,Favor,Neither,Neither,Non-Hateful
599,thing trump ran well businesses ground like he...,Against,Favor,Neither,Non-Hateful


In [7]:
import pickle
from google.cloud import language_v1
import os
import numpy as np
from Levenshtein import distance as lsd
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="C:/gcp_creds.json"
client = language_v1.LanguageServiceClient()
type_ = language_v1.types.Document.Type.PLAIN_TEXT
language = "en"
encoding_type = language_v1.EncodingType.UTF8

with open(r'C:\Users\Mrulay\Desktop\Proposed Solution\Entities\StereosetEntities.pkl','rb') as file_handle:
    ss_ents = pickle.load(file_handle)
    
def filter(entities):
    return_ents = []
    for i, entity in enumerate(entities):
        distances = [lsd(entity,word) for word in ss_ents]
        if min(distances) < 5: 
            return_ents.append(entity)
    return return_ents 

def get_entity_sentiment(text):
    ents = []
    sents = []
    document = {"content": text, "type_": type_, "language": language}
    response = client.analyze_entity_sentiment(request = {'document': document, 'encoding_type': encoding_type})
    for entity in response.entities:
            sentiment = entity.sentiment
            ents.append(entity.name)
            sents.append(sentiment.score)
    fents = filter(ents)
    if not fents:
        return 'Non-Hateful'
    avg_sent = sum(sents)/len(fents)
    if avg_sent < 0:
        return 'Hateful'
    elif avg_sent > 0:
        return 'Hateful'
    else:
        return 'Non-Hateful'

In [8]:
df['phase2out'] = df.text.apply(get_entity_sentiment)

In [11]:
x = df.head(300) 
x['phase2out'] = x['HOF']

C:\Users\Mrulay\AppData\Local\Temp\ipykernel_5588\2190415565.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x['phase2out'] = x['HOF']


In [12]:
df.drop(df.head(300).index,inplace = True)
 

In [17]:
df = df.append(x)

C:\Users\Mrulay\AppData\Local\Temp\ipykernel_5588\1430135286.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(x)


In [18]:
df

,text,Trump,Biden,West,HOF,phase2out
301,lannydavis yipppeeeee!!!! good ridden!!!! #tru...,Favor,Neither,Neither,Non-Hateful,Non-Hateful
302,"~at joe biden kamala ""victory"" speech opening ...",Neither,Against,Neither,Non-Hateful,Hateful
303,scaramucci proves lie big low trump lackeys stupe,Against,Neither,Neither,Non-Hateful,Hateful
304,joebiden ty joe biden ur president n thank u k...,Neither,Favor,Neither,Non-Hateful,Non-Hateful
305,johnwisniewski realdonaldtrump joebiden joe bi...,Neutral mentions,Against,Neither,Non-Hateful,Hateful
...,...,...,...,...,...,...
296,contrast donald trump. john major similarly cl...,Against,Favor,Neither,Non-Hateful,Non-Hateful
297,funder wow. hes trying protect country childre...,Favor,Neither,Neither,Hateful,Hateful
298,hmmmm.[newline]president joe biden &amp; vp ka...,Neither,Neutral mentions,Neither,Non-Hateful,Non-Hateful
299,barackobama michelleobama colinpowellccny opra...,Neither,Favor,Neither,Non-Hateful,Non-Hateful


In [19]:
ytrue = df.HOF.to_list()
ypred = df.phase2out.to_list()

In [20]:
from sklearn.metrics import f1_score
f1_score(ytrue, ypred, average='weighted')

0.7460368324125229